In [ ]:
import sqlite3
import pandas as pd
import re
import numpy as np
import os
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats


In [ ]:
db_path = "data/agri.db"  
conn = sqlite3.connect(db_path)

Show all tables in the database

In [ ]:
query = "SELECT * FROM farm_data;"
df = pd.read_sql(query, conn)

print(df)

1. Preparing the Data

In [ ]:
df.dtypes

Cleaning and preparing the data

In [ ]:
remove_ppm = ["Nutrient N Sensor (ppm)", "Nutrient P Sensor (ppm)", "Nutrient K Sensor (ppm)"]

for col in remove_ppm:
    df[col] = (
        df[col]
        .astype(str)  # Convert all values to string
        .str.replace("ppm", "", regex=False)  # Remove 'ppm'
        .str.strip()  # Remove whitespace
        .replace(["", "None", "nan"], pd.NA)  # Replace empty strings and 'None' with Pandas NA
        .astype("Int64")  # Convert to integer while keeping NaNs
    )

df.dtypes


In [ ]:
object_columns = ["System Location Code","Previous Cycle Plant Type","Plant Type", "Plant Stage"]

for col in object_columns:
    df[col] = df[col].astype(str).str.strip().str.title()
    df[f"new_{col}"] = pd.factorize(df[col])[0]  # Assign unique integer values

df[[f"new_{col}" for col in object_columns]] = df[[f"new_{col}" for col in object_columns]].astype("int64")
df.dtypes

Change object type into int type for analysis

In [ ]:
df_new = df.drop(columns=object_columns)
df_new.dtypes


Checking for missing values percentage

In [ ]:
for col in df_new.columns:
    missing_percentage = (df_new[col].isnull().sum() / len(df_new)) * 100
    print(f"Percentage of missing values in '{col}': {missing_percentage:.2f}%")

We can see that column "Humidity Sensor (%)" has a missing value percentage of 67.61% which is quite high. The column might be irrelvant for analysis.
For columns like "Temperature Sensor (%)", "Light Intensity Sensor (lux)", "Nutrient N Sensor (ppm)", "Nutrient K Sensor (ppm)", " Nutrient P Sensor (ppm)" and Water Levl Sensor (mm)", we can either fill in with estimated values or removed the rows.

check correlation between Humidity Sensor and Temperature Sensor

In [ ]:
corr_temp_humidity = df_new['Temperature Sensor (°C)'].corr(df['Humidity Sensor (%)'])
print(f"Correlation (Temp vs Humidity): {corr_temp_humidity:.2f}")

check correlation between Humidity Sensor and Plant-Type-Stage

In [ ]:
# Combine "Plant Type" and "Plant Stage" into a new feature
df_new["Plant Type-Stage"] = df_new["new_Plant Type"] * 10 + df_new["new_Plant Stage"]  # Unique encoding

# Compute correlation
correlation = df_new[["Humidity Sensor (%)", "Plant Type-Stage"]].corr()

# Print correlation matrix
print("Correlation between Humidity Sensor (%) and Plant Type-Stage:")
print(correlation)

We can see that humidity has low correalation with Temperature Sensor and Plant Type/Plant Stage, hence we can drop the coloumn.

In [ ]:
df_new = df_new.drop(columns="Humidity Sensor (%)")

print(df_new)

In [ ]:
df_new.dtypes

In [ ]:
print(df_new)

In [ ]:
numeric_columns = df_new.select_dtypes(include=['int64', 'float64']).columns

# Calculate correlations with Temperature Sensor using only numeric columns
correlations = df_new[numeric_columns].corr()['Temperature Sensor (°C)'].sort_values(ascending=False)

# Print the correlations
print(correlations)

# Create a visualization using only numeric columns
plt.figure(figsize=(12, 8))
sns.heatmap(df_new[numeric_columns].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix (Numeric Columns Only)')
plt.tight_layout()
plt.show()

# Bar chart for Temperature correlations
plt.figure(figsize=(12, 6))
correlations.drop('Temperature Sensor (°C)', errors='ignore').plot(kind='bar')
plt.title('Correlation with Temperature Sensor')
plt.ylabel('Correlation Coefficient')
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.tight_layout()
plt.show()

I will now fix the missing values of the data and test which method will increase the correlation.

In [ ]:
for col in df_new.columns:
    missing_percentage = (df_new[col].isnull().sum() / len(df_new)) * 100
    print(f"Percentage of missing values in '{col}': {missing_percentage:.2f}%")

In [ ]:
fill_up = ["Temperature Sensor (°C)", "Light Intensity Sensor (lux)", "Nutrient N Sensor (ppm)", "Nutrient P Sensor (ppm)", "Nutrient K Sensor (ppm)", "Water Level Sensor (mm)"]

df_filled = df_new.copy()  # Use parentheses to call the copy method

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Uses 5 nearest neighbors

# Apply KNN imputation only to the selected columns in the new DataFrame
df_filled[fill_up] = knn_imputer.fit_transform(df_filled[fill_up])

# Print the new DataFrame
print(df_filled)

In [ ]:
#check if all columns are filled

for col in df_filled.columns:
    missing_percentage = (df_filled[col].isnull().sum() / len(df_filled)) * 100
    print(f"Percentage of missing values in '{col}': {missing_percentage:.2f}%")


In [ ]:
# Calculate correlations with Temperature Sensor using only numeric columns
correlations = df_filled[numeric_columns].corr()['Temperature Sensor (°C)'].sort_values(ascending=False)

# Print the correlations
print(correlations)

# Create a visualization using only numeric columns
plt.figure(figsize=(12, 8))
sns.heatmap(df_filled[numeric_columns].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix (Numeric Columns Only)')
plt.tight_layout()
plt.show()

# Bar chart for Temperature correlations
plt.figure(figsize=(12, 6))
correlations.drop('Temperature Sensor (°C)', errors='ignore').plot(kind='bar')
plt.title('Correlation with Temperature Sensor')
plt.ylabel('Correlation Coefficient')
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.tight_layout()
plt.show()

We can see that by filling up the rows with missing valuers, the correlation actually decreased. Hence, I will check if removing rows with the missing values would improve the correlation instead.

In [ ]:
df_new = df_new.dropna()

for col in df_new.columns:
    missing_percentage = (df_new[col].isnull().sum() / len(df_new)) * 100
    print(f"Percentage of missing values in '{col}': {missing_percentage:.2f}%")

In [ ]:
correlations = df_new[numeric_columns].corr()['Temperature Sensor (°C)'].sort_values(ascending=False)

# Print the correlations
print(correlations)

# Create a visualization using only numeric columns
plt.figure(figsize=(12, 8))
sns.heatmap(df_new[numeric_columns].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix (Numeric Columns Only)')
plt.tight_layout()
plt.show()

# Bar chart for Temperature correlations
plt.figure(figsize=(12, 6))
correlations.drop('Temperature Sensor (°C)', errors='ignore').plot(kind='bar')
plt.title('Correlation with Temperature Sensor')
plt.ylabel('Correlation Coefficient')
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print(df_new)

We can see that the correlation improved slightly. Hence, I will be removing rows with missing values.

Now I will be using different models and test which model performs better then creating a predictive model for temperature condition.

In [ ]:
# 1. Prepare features and target
X = df_new.drop('Temperature Sensor (°C)', axis=1)  # All other columns as features
y = df_new['Temperature Sensor (°C)']               # Temperature as target

# 2. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Scale the features (important for many ML algorithms)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}


results = {}
feature_importances = {}

for name, model in models.items():
    # Train the model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # MAPE in percentage
    
    results[name] = {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}
    
    # Store feature importances if available
    if hasattr(model, 'feature_importances_'):
        feature_importances[name] = pd.Series(
            model.feature_importances_,
            index=X.columns
        ).sort_values(ascending=False)
    elif hasattr(model, 'coef_'):
        feature_importances[name] = pd.Series(
            np.abs(model.coef_),
            index=X.columns
        ).sort_values(ascending=False)


In [ ]:
print("Model Performance:")
for name, metrics in results.items():
    print(f"{name}: MSE = {metrics['MSE']:.4f}, RMSE = {metrics['RMSE']:.4f}, MAE = {metrics['MAE']:.4f}, R2 = {metrics['R2']:.4f}, MAPE = {metrics['MAPE']:.2f}%")

# Identify the best model based on R2 score
best_model_name = max(results, key=lambda x: results[x]['R2'])
print(f"\nBest model: {best_model_name} with R2 = {results[best_model_name]['R2']:.4f}")

# Plot feature importances if available
if best_model_name in feature_importances:
    plt.figure(figsize=(12, 8))
    feature_importances[best_model_name].plot(kind='bar', color='skyblue')
    plt.title(f'Feature Importances - {best_model_name}')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.tight_layout()
    plt.show()

# Cross-validation for the best model
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"\nCross-Validation R2 Scores: {cv_scores}")
print(f"Mean Cross-Validation R2 Score: {cv_scores.mean():.4f}")

# Confidence interval for R2
confidence = 0.95
ci = stats.norm.interval(confidence, loc=cv_scores.mean(), scale=cv_scores.std() / np.sqrt(len(cv_scores)))
print(f"95% Confidence Interval for R2: {ci}")

# Residual analysis
y_pred = best_model.predict(X_test_scaled)
residuals = y_test - y_pred

plt.figure(figsize=(12, 6))
plt.scatter(y_pred, residuals, alpha=0.5, color='blue')
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Temperature (°C)')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# Predicted vs Actual plot with different colors
plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_test, color='blue', alpha=0.5, label='Actual')  # Actual values in blue
plt.scatter(y_test, y_pred, color='orange', alpha=0.5, label='Predicted')  # Predicted values in orange
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Ideal Prediction')  # Ideal line
plt.xlabel('Actual Temperature (°C)')
plt.ylabel('Predicted Temperature (°C)')
plt.title('Predicted vs Actual Temperature')
plt.legend(loc='best')
plt.show()

# Learning curves
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_train_scaled, y_train, cv=5, scoring='r2', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(12, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='r')
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color='g')
plt.title('Learning Curves')
plt.xlabel('Training Set Size')
plt.ylabel('R2 Score')
plt.legend(loc='best')
plt.show()